In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
# from kernels.squared_exponential import SquaredExponential
# from kernels.matern import Matern
from torch.optim import Adam

# import sys
import math
# sys.path.append('/Users/colecitrenbaum/Documents/GPs/gp-quadrature/Tests and Sanity Checks/')
from efgpnd import EFGPND
import warnings
# warnings.filterwarnings("ignore", message=".*disabling cuda.*")


# Generating some synthetic data

In [2]:


# --- Parameters ---
n = 10_000  # Number of points
d = 2  # Dimensionality of the input space
true_length_scale =0.15
true_variance = 1
true_noise_variance = 0.2
dtype = torch.float32  # Use float64 as in the original example
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available
print(f"Using device: {device}")

# --- Generate Input Points ---
# Generate random points in d-dimensional space from -1 to 1
x = torch.rand(n, d, dtype=dtype, device=device) * 2 - 1




Using device: cpu


In [3]:
# Generate test points (grid of points for visualization if d <= 3)
if d >= 2:
    # Create a grid of points for testing
    grid_points_per_dim = 20
    grid_points = [torch.linspace(x.min()-0.1, 0.1+x.max(), grid_points_per_dim, dtype=dtype, device=device) for _ in range(d)]
    mesh_grid = torch.meshgrid(*grid_points, indexing='ij')
    x_new = torch.stack([grid.flatten() for grid in mesh_grid], dim=1)
elif d==1:
    grid_points_per_dim = 3000
    grid_points = [torch.linspace(x.min()-0.1, 0.1+x.max(), grid_points_per_dim, dtype=dtype, device=device) for _ in range(d)]
    mesh_grid = torch.meshgrid(*grid_points, indexing='ij')
    x_new = torch.stack([grid.flatten() for grid in mesh_grid], dim=1)
else:
    # For higher dimensions, just use random test points
    x_new = torch.rand(1000, d, dtype=dtype, device=device) * 2.4 - 1.2

In [4]:
from vanilla_gp_sampling import sample_gp_fast, sample_gp_matern

# # For squared exponential kernel
samples_se = sample_gp_fast(
    x,
    length_scale=true_length_scale,
    variance=true_variance,
    noise_variance=true_noise_variance
)
# # For Matern kernel
# samples_m32 = sample_gp_matern(
#     x,
#     nu=1.5,  # 3/2 Matern
#     length_scale=true_length_scale,
#     variance=true_variance,
#     noise_variance=true_noise_variance
# )
y = samples_se

# Using EFGPND 

In [5]:
d= 2 
EPSILON = 1e-4 # bound on kernel error 
cg_tol = EPSILON 

## Hyperparameter learning
    Note I hooked optimizer.step() to sync_parameters() so that the kernel is updated after each step, so that compute gradients is always called with the latest kernel hyperparameters.
    Also, compute_gradients by default puts the grads in model._gp_params.hyper.rawgrad so that optimizer can use them.

In [6]:
max_iters = 50
J = 5
## hyper learning with Adam
model = EFGPND(x, y, kernel="SquaredExponential", eps=EPSILON)
# params = next(model.parameters())

optimizer = Adam(model.parameters(), lr=0.1)
for it in range(max_iters):
    optimizer.zero_grad()
     # saves grads in model._gp_params so that we can step 
    model.compute_gradients(
                trace_samples=J,
            )

    optimizer.step() 
    


    if it % 10 == 0:
        lengthscale = model.kernel.get_hyper('lengthscale')
        variance = model.kernel.get_hyper('variance')
        sigmasq = model._gp_params.sig2.item()
        # Get current values for printing
        print(f"[ε={EPSILON} | J={J}] iter {it:>3}  "
            f"ℓ={lengthscale:.4g}  "
            f"σ_f²={variance:.4g}  σ_n²={sigmasq:.4g}")

print(f'Final hyperparams: ℓ={lengthscale:.4g}, σ_f²={variance:.4g}, σ_n²={sigmasq:.4g}')


[ε=0.0001 | J=5] iter   0  ℓ=0.2976  σ_f²=0.819  σ_n²=0.1042
[ε=0.0001 | J=5] iter  10  ℓ=0.1452  σ_f²=1.676  σ_n²=0.2244
[ε=0.0001 | J=5] iter  20  ℓ=0.12  σ_f²=1.817  σ_n²=0.2554
[ε=0.0001 | J=5] iter  30  ℓ=0.1266  σ_f²=1.387  σ_n²=0.205
[ε=0.0001 | J=5] iter  40  ℓ=0.1434  σ_f²=0.9796  σ_n²=0.1831
Final hyperparams: ℓ=0.1434, σ_f²=0.9796, σ_n²=0.1831


## Fitting posterior mean

## "fit" is confusing here
- Context variables instead of all the opts 
- with .... settings = exact... 

In [7]:
# Time different variance estimation methods
import time

# No variance
start_time = time.time()
mean_no_var, _= model.predict(x_new, return_variance=False)
no_var_time = time.time() - start_time
print(f"Time without variance: {no_var_time:.4f} seconds; x_new.shape = {x_new.shape}")


Time without variance: 0.0412 seconds; x_new.shape = torch.Size([400, 2])


## Posterior variance -- Stochastic Estimate

In [8]:
start_time = time.time()
hutchinson_probes = 100
mean, stoch_var = model.predict(x_new, return_variance=True, variance_method="stochastic", hutchinson_probes=hutchinson_probes)
stoch_var_time = time.time() - start_time
print(f"Time with stochastic variance, x_new.shape = {x_new.shape}, {hutchinson_probes} probes: {stoch_var_time:.4f} seconds")




Time with stochastic variance, x_new.shape = torch.Size([400, 2]), 100 probes: 0.7475 seconds


## Posterior variance -- regular method

In [9]:
# Regular variance
start_time = time.time()
mean, var = model.predict(x_new, return_variance=True, variance_method="regular")
reg_var_time = time.time() - start_time
print(f"Time with regular variance, x_new.shape = {x_new.shape}: {reg_var_time:.4f} seconds")

Time with regular variance, x_new.shape = torch.Size([400, 2]): 2.5758 seconds
